In [1]:
import pandas as pd

In [ ]:
df = pd.read_json("./runtimes/toxicainment_videos_log.jsonl", lines=True)
df.head(2)

In [ ]:
cdf = (
    df.groupby("Run_ID", as_index=False)["Timestamp"]
    .min()
    .sort_values("Timestamp")
    .assign(concurrency=[1, 4, 16, 50, 8, 2, 32, 12])
)
cdf

In [4]:
begin = df.groupby("Run_ID", as_index=False).agg(
    t0=("Timestamp", "first"), d0=("Total_Runtime", "first")
)
df = pd.merge(df, begin, on="Run_ID", how="left").assign(
    tx=lambda df: (df["Timestamp"] - df["t0"]).dt.total_seconds() + df["d0"]
)

In [ ]:
df = pd.merge(df, cdf[["Run_ID", "concurrency"]], on="Run_ID", how="left")
df.head(2)

In [ ]:
sdf = df.groupby("concurrency", as_index=False).agg(
    avg_runtime=("Total_Runtime", "mean"),
    total_runtime=("Total_Runtime", "sum"),
    first_response=("Total_Runtime", "first"),
    n_requests=("Total_Runtime", "count"),
)
sdf

In [ ]:
import seaborn as sns

sns.scatterplot(data=df, x="tx", y="Total_Runtime", hue="concurrency", hue_norm=None, palette="deep")

In [ ]:
ax = sns.scatterplot(sdf, x="concurrency", y="total_runtime")
ax.grid(alpha=0.5)
ax.set_ylim(0, None)